# Semantische Ähnlichkeit von Texten

Natürliche Sprachen bieten uns mannigfaltige Möglichkeiten, dieselben Inhalte auf unterschiedliche Art und Weise auszudrücken. Für die Interpretation natürlichsprachlicher Texte reicht es daher nicht, auf den Ebenen von Morphologie und Syntax zu bleiben und die Form von Wörtern sowie deren Anordnung im Satz zu betrachten, sondern wir müssen als zusätzliche Ebene die der Semantik betrachten.

Dabei beginnen wir zunächst mit der Betrachtung der Wortebene und sammeln Erfahrung mit word2vec.

Anschließend wollen wir versuchen, von Wort- zu Satzbedeutungen zu kommen: Wie gut können wir die Ähnlichkeit zweier Sätze unter Rückgriff auf die semantischen Embeddings der in ihnen enthaltenen Wörter bestimmen?

Als Orientierung dient uns dabei das [STS-Benchmark-Datenset](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark), das im Rahmen von [SemEval](https://en.wikipedia.org/wiki/SemEval) entstanden ist.
Das Datenset enthält Trainings-, Test- und Evaluationsdaten zu folgender Aufgabenstellung: 
Gegeben zwei Sätze S1 und S2, berechne Ähnlichkeitswert zwischen 0 (völlig unterschiedlicher Inhalt) und 5 (bedeutungsgleich).

## Aufgabe 1: Word2Vec
> You shall know a word by the company it keeps.
>
> -- <cite>J. R. Firth</cite>

Im ersten NLP-Labor haben wir mit dem Bag-of-Words-Modell eine recht einfache Methode festgestellt, um Wörter bzw. ganze Sätze in einen Vektorraum abzubilden: Jedem Wort wird ein eindeutiger Index zugewiesen. Es lässt sich dann als one-hot-enkodierter Vektor darstellen, der an eben jenem Index eine Eins und ansonsten nur Nullen enthält und dessen Länge der Anzahl der Wörter im Vokabular entspricht. Die Enkodierung ganzer Sätze erhält man durch Aufsummieren dieser one-hot-enkodierten Vektoren.
Bei einem solchen Modell geht auf Satzebene die Information über die Reihenfolge der Wörter verloren und ganz allgemein wird offensichtlich die Semantik der Wörter nicht berücksichtigt.

Gehen wir zum Beispiel von folgendem Vokabular aus: ```{"heiß", "warm", "Gurke"}``` und enkodieren die einzelnen Wörter wie folgt: ```[1, 0, 0], [0, 1, 0], [0, 0, 1]```, dann sind sich "heiß" und "warm" genauso ähnlich oder unähnlich wie "heiß" und "Gurke".

Semantische Embeddings schaffen an dieser Stelle Abhilfe. Ein bekannter Vertreter dieser Ansätze ist das Word2Vec-Modell, das 2013 von [Tomas Mikolov et al.](https://arxiv.org/abs/1301.3781) bei Google entwickelt wurde und seither rasch an Verbreitung gewonnen hat.

Wir werden mit vortrainierten Google-News-Embeddings arbeiten, die [hier](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing) heruntergeladen werden können. Die Vektoren haben die Länge 300.

### Aufgabe 1.1: Vorbereitung
Ladet die Embeddings über den oben angegebenen Link herunter und verwendet gensim, um sie zu laden.

**Hinweis**: Kann einen Moment dauern, bis das Dictionary, das von Wort auf Embedding abbildet, erzeugt ist. Im Zweifel ein ```limit``` angeben und nur die ersten 1,5 Mio. Embeddings laden.

In [3]:
import gensim

embeddings = gensim.models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin', binary=True, limit=2000000)

In [4]:
def check_embeddings(embeddings):
    error_message = "Oh, da ist wohl etwas schiefgelaufen! Der folgende Test schlägt fehl: {}."
    most_similar = embeddings.most_similar(positive=['man', 'queen'], negative=['woman'])
    if len(most_similar) < 1 or most_similar[0][0] != 'king':
        return error_message.format("'Most similar'")

    doesnt_match = embeddings.doesnt_match(['spring', 'rain', 'autumn', 'summer'])
    if doesnt_match != 'rain':
        return error_message.format("'Doesn't match'")
    
    most_similar_to_given = embeddings.most_similar_to_given('school', ['cat', 'sound', 'university', 'whine'])
    if most_similar_to_given != 'university':
        return error_message.format("'Most similar to given'")
    
    return "Sieht gut aus."

check_embeddings(embeddings)

/anaconda3/envs/lab/lib/python3.6/site-packages/gensim/models/keyedvectors.py:730: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'Sieht gut aus.'

### Aufgabe 1.2: Spaß mit Semantik
Im Folgenden wollen wir uns ein bisschen mit dem Mehrwert beschäftigen, den semantische Embeddings bieten.

Mit ihnen lassen sich zum Beispiel folgende Fragen beantworten:


In [24]:
# Welche Stadt ist das New York Deutschlands? (Hinweis: 'New_York' ist als Token in den Embeddings enthalten)
print('Das deutsche New York ist: {}'.format(embeddings.most_similar_cosmul(positive=['New_York', 'Germany'], negative=['USA'])[0][0]))

# Wer ist eigentlich der Mozart der Naturwissenschaft?
print('Der Mozart der Naturwissenschaft ist: {}'.format(embeddings.most_similar_cosmul(positive=['Mozart', 'science'], negative=['music'])[0][0]))

# Welches Wort verhält sich zu 'singing' wie 'burnt' zu 'burning'?
print('burning:burnt wie singing:{}'.format(embeddings.most_similar_cosmul(positive=['singing', 'burnt'], negative=['burning'])[0][0]))

# Sind sich Deutschland und Frankreich ähnlicher oder Deutschland und Kanada?
print('Ähnlichkeit DE, FR: {}'.format(embeddings.similarity('Germany', 'France')))
print('ähnlichkeit DE, CAN: {}'.format(embeddings.similarity('Germany', 'Canada')))

Das deutsche New York ist: Berlin
Der Mozart der Naturwissenschaft ist: Gottfried_Wilhelm_Leibniz
burning:burnt wie singing:sang
Ähnlichkeit DE, FR: 0.6270756125450134
ähnlichkeit DE, CAN: 0.3807849586009979


Bei der Interpretation der Ergebnisse ist jedoch Vorsicht geboten: Es werden zwar semantische Beziehungen abgebildet, aber die entsprechen möglicherweise nicht immer den Erwartungen.

Wie ähnlich sind sich zum Beispiel "Leben" und "Tod", "kalt" und "warm", "Norden" und "Süden"?

Sind die Ergebnisse wie erwartet? Warum (nicht)?

In [25]:
print(embeddings.similarity("life", "death"))
print(embeddings.similarity("cold", "warm"))
print(embeddings.similarity("North", "South"))

0.36187765
0.59530354
0.7784994


Die Embeddings sind nicht neutral, sondern spiegeln die Beziehungen wieder, die sich in den Trainingsdaten finden lassen.

In [26]:
# Wird Wissenschaft von Frauen oder Männern gemacht?
print('Wissenschaft wird gemacht von: {}'.format(embeddings.most_similar_to_given('scientist', ['man', 'woman'])))
# Sind Mörder eher Schwarze, Weiße oder Asiaten?
print('Mörder sind: {}'.format(embeddings.most_similar_to_given('killer', ['black', 'white', 'asian'])))
# Was bleibt vom Mann, wenn die Intelligenz abgezogen wird?
print('Mann ohne Intelligenz: {}'.format(embeddings.most_similar(positive=['man'], negative=['intelligence'])[0][0]))


Wissenschaft wird gemacht von: man
Mörder sind: black
Mann ohne Intelligenz: woman


## Aufgabe 2: Von Wörtern zu Sätzen
Nachdem wir uns mit der semantischen Repräsentation von Wörtern beschäftigt haben, wollen wir jetzt zu Sätzen übergehen. Wie bereits gesagt, wollen wir die Ähnlichkeit von Satzpaaren bestimmen. Dazu brauchen wir einen Testdatensatz.

### Aufgabe 2.1: Testdaten einlesen
Die Datei ```sts-train.csv``` enthält 5749 Satzpaare, deren Ähnlichkeit jeweils mit einem Score zwischen 0 (völlig unähnlich) und 5 (völlig ähnlich) bewertet wurde. Lest diese Datei in einen Pandas-Dataframe ein.

Zu beachten sind die folgenden Punkte:
* Bei den CSV-Dateien handelt es sich eigentlich strenggenommen um TSV-Dateien, d.h. die einzelnen Spalten sind durch Tab getrennt.
* Einige Zeilen enthalten unvollständige Quotes, die zu Fehlern beim Einlesen führen können. Diese sollen ignoriert werden. (```quoting=csv.QUOTE_NONE```)
* Zitat aus der zum Datenset gehörigen README: 
>Each file is encoded in utf-8 (a superset of ASCII), and has the following tab separated fields:  
>__genre filename year score sentence1 sentence2__  
>optionally there might be some license-related fields after sentence2.

Neben den uns interessierenden Spalten "score", "sentence1" und "sentence2" sind also noch weitere Spalten enthalten. Wir können den Parameter ```usecols``` verwenden, um die Indizes der uns interessierenden Spalten anzugeben (Zählung beginnt bei 1). Für die Spalten vergeben wir die Namen "score", "sentence1" und "sentence2".

In [5]:
import pandas as pd
import csv

testdata =  pd.read_csv('./data/sts-train.csv', sep='\t', quoting=csv.QUOTE_NONE, usecols=[4, 5, 6], names = ["score", "sentence1", "sentence2"])


In [6]:
testdata

,score,sentence1,sentence2
0,5.000,A plane is taking off.,An air plane is taking off.
1,3.800,A man is playing a large flute.,A man is playing a flute.
2,3.800,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.600,Three men are playing chess.,Two men are playing chess.
4,4.250,A man is playing the cello.,A man seated is playing the cello.
5,4.250,Some men are fighting.,Two men are fighting.
6,0.500,A man is smoking.,A man is skating.
7,1.600,The man is playing the piano.,The man is playing the guitar.
8,2.200,A man is playing on a guitar and singing.,A woman is playing an acoustic guitar and sing...
9,5.000,A person is throwing a cat on to the ceiling.,A person throws a cat on the ceiling.


### Aufgabe 2.2: Semantische Repräsentation von Sätzen
Genau wie für Wörter wollen wir auch die Semantik von Sätzen in einem Vektor der Dimension 300 abbilden. Dazu wählen wir einen recht einfachen Ansatz und mitteln die Vektoren der Wörter, aus denen der Satz besteht.

Schreibt eine Funktion, die einen Satz entgegennimmt und einen Vektor mit den gemittelten semantischen Embeddings zurückgibt. Wird ein leerer String übergeben oder enthält der übergebene Satz nur Wörter, für die keine Embeddings vorhanden sind, soll ein Nullvektor der Länge 300 zurückgegeben werden.

In [43]:
import numpy as np
from string import punctuation

def process_word(word):
    word = word.lower()
    word = word.strip(punctuation)
    return word

def sentence_to_vec(sentence):
    sentence = [process_word(word) for word in sentence.split() if process_word(word) in embeddings.vocab]
    vectors = [embeddings[word] for word in sentence]
    if len(vectors) == 0:
        return np.zeros(300)
    return np.mean(vectors, axis=0)

In [44]:
def sentence_to_vec_tests():
    if (np.zeros(300) != sentence_to_vec('')).any():
        return "Bei leerer Eingabe soll ein Nullvektor zurückgegeben werden"
    if (np.zeros(300) != sentence_to_vec('thereisnosuchword')).any():
        return "Wenn keine Embeddings gefunden werden, soll ein Nullvektor zurückgegeben werden."
    if (embeddings['word'] != sentence_to_vec('word')).any():
        return "Da stimmt was nicht."
    if ((embeddings['cat'] + embeddings['dog']) / 2 != sentence_to_vec('cat dog')).any():
        return "Die Funktion soll den Durchschnittswert der Vektoren berechnen."
    if (embeddings['word'] != sentence_to_vec('thereisnosuchword word')).any():
        return "Wörter, deren Embedding nicht bekannt ist, sollen nicht berücksichtigt werden."
    return "Grundlegende Tests bestanden."

In [45]:
print(sentence_to_vec_tests())

Grundlegende Tests bestanden.


### Aufgabe 2.3: Ähnlichkeitsberechnung
Um die Ähnlichkeit zweier Vektoren zu bestimmen, wird häufig die [Kosinusähnlichkeit](https://en.wikipedia.org/wiki/Cosine_similarity) verwendet. Weil wir die von uns berechneten Ähnlichkeitswerte aber mit denen der Testdaten vergleichen können wollen, müssen wir den Wertebereich so anpassen, dass wir Werte zwischen 0 und 5 erhalten. Dabei treffen wir die Annahme, dass eine Kosinusähnlichkeit kleiner-gleich 0 als unähnlich (Wert ```0```) zu werten ist.

Schreibt eine Funktion, die zwei Sätze entgegennimmt und Ähnlichkeitswerte im Bereich ```[0, 5]``` zurückgibt, basierend auf der Kosinusähnlichkeit der semantischen Vektoren der Sätze.

In [46]:
from sklearn.metrics.pairwise import cosine_similarity
def compute_similarity_score(sentence_1, sentence_2):
    cos_similarity = cosine_similarity(sentence_to_vec(sentence_1).reshape(1, -1), sentence_to_vec(sentence_2).reshape(1, -1))
    our_similarity = max(0.0, cos_similarity[0][0]) * 5
    return our_similarity

In [47]:
import math
def compute_similarity_score_test():
    if not (math.isclose(compute_similarity_score("hot dog", "hot dog"), 5.0)):
        return "Identische Sätze sollten einen Ähnlichkeitswert von 5 haben."
    if not (math.isclose(compute_similarity_score("dog hot", "hot dog"), 5.0)):
        return "Sätze, die sich nur in der Anordnung der Wörter unterscheiden, sollten einen Ähnlichkeitswert von 5 haben."
    if not (math.isclose(compute_similarity_score("nosuchword", "word"), 0.0)):
        return "Der Vergleich eines unbekannten Wortes mit einem bekannten, sollte zu einem Ähnlichkeitswert von 0 führen."
    if (math.isclose(compute_similarity_score("bread", "cake"), 0.0)):
        return "Verwandte Worte sollten einen Ähnlichkeitswert größer Null aufweisen."
    return "Grundlegende Tests bestanden."

compute_similarity_score_test()

'Grundlegende Tests bestanden.'

## Aufgabe 3: Validierung
Im letzten Schritt wollen wir bestimmen, wie gut unsere Ähnlichkeitsberechnung funktioniert, d.h. wie stark die von uns berechneten Ähnlichkeitswerte für die Satzpaare aus dem Testdatensatz mit den erwarteten Werten übereinstimmen.

Dazu verwenden wir den [Korrelationskoeffizienten](https://de.wikipedia.org/wiki/Korrelationskoeffizient), der uns in ```scipy``` als [```stats.pearsonr()```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html) zur Verfügung steht.

Schreibt eine Funktion, die zwei Listen mit Ähnlichkeitswerten entgegennimmt und für diese den Pearsonscore berechnet. Wendet die Funktion auf die für die Testdaten erwarteten und die von uns berechneten Ähnlichkeitsscores an und interpretiert das Ergebnis.

In [48]:
import numpy as np
from scipy import stats
def calculate_pearson_score(expected, actual):
    return stats.pearsonr(expected, actual)

In [49]:
expected_scores = testdata['score'].to_numpy()
our_scores = testdata.apply(lambda row: compute_similarity_score(row['sentence1'], row['sentence2']), axis=1)
    
print(calculate_pearson_score(expected_scores, our_scores))


(0.6821517528724196, 0.0)
